# CS6330 Project 1: Reinforcement Learning Blackjack Agent

In [1]:
import blackjack

In [2]:
blackjack.Game().play_hand()

{'winner': 'dealer',
 'player_score': 19,
 'dealer_score': 20,
 'player_cards': [<Rank.SEVEN: 7>, <Rank.EIGHT: 8>, <Rank.FOUR: 4>],
 'dealer_cards': [<Rank.NINE: 9>, <Rank.SIX: 6>, <Rank.FIVE: 5>]}